# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2023spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/15lGUmT8NpLGtoxRllRWCJyQEjhR1Idcei63YHsDckPE/edit#slide=id.g21fff4e9af6_0_13)　Kaggle: [Link](https://www.kaggle.com/competitions/ml2023spring-hw7/host/sandbox-submissions)　Data: [Link](https://drive.google.com/file/d/1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7/view?usp=sharing)




# Prerequisites

## Download Dataset

In [1]:
# download dataset by youself!
# unzip dataset
!unzip -o hw7_data.zip

unzip:  cannot find or open hw7_data.zip, hw7_data.zip.zip or hw7_data.zip.ZIP.


## Install packages

Documentation for the toolkit:　https://huggingface.co/transformers/

In [2]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.26.1
!pip install accelerate==0.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 5.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


# Kaggle (Fine-tuning)

## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Import Packages

In [6]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(24)

## Load Model and Tokenizer




 

In [4]:
from transformers import (
  AutoTokenizer,
  AutoModelForQuestionAnswering,
  get_linear_schedule_with_warmup
)

# model = AutoModelForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large").to(device)
# tokenizer = AutoTokenizer.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")
model = AutoModelForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large").to(device)
tokenizer = AutoTokenizer.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")
#luhua/chinese_pretrain_mrc_roberta_wwm_ext_large
#deepset/roberta-base-squad2
#luhua/chinese_pretrain_mrc_macbert_large
# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26918 QA pairs
- Dev set: 2863  QA pairs
- Test set: 3524  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [7]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

# Change the path of the dataset
train_questions, train_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_train.json")
dev_questions, dev_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_dev.json")
test_questions, test_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_test.json")

In [8]:
#參考:https://github.com/pai4451/ML2021/tree/main/hw7
print(tokenizer.tokenize('♡', add_special_tokens=False))
print(tokenizer.tokenize('△', add_special_tokens=False))
print(tokenizer.tokenize('□', add_special_tokens=False))
print(tokenizer.tokenize('○', add_special_tokens=False))
print(tokenizer.tokenize('*', add_special_tokens=False))
#train_paragraphs = [i.replace(' ','✔').replace('\u200b','✦').replace('\u200e', '☺').replace('\u3000', '☆').replace('#','●') for i in train_paragraphs]
dev_paragraphs = [i.replace('“',' ').replace('”',' ').replace(' ','♡').replace('\u200b','△').replace('\u200e', '□').replace('\u3000', '○').replace('#','*') for i in dev_paragraphs]
test_paragraphs = [i.replace('“',' ').replace('”',' ').replace(' ','♡').replace('\u200b','△').replace('\u200e', '□').replace('\u3000', '○').replace('#','*') for i in test_paragraphs]

['♡']
['△']
['□']
['○']
['*']


## Tokenize Data

In [9]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False, max_length=512)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


## Dataset

In [10]:
#參考資料 :https://blog.csdn.net/qq_43613342/article/details/127044475?spm=1001.2014.3001.5502
#參考資料 :https://zhuanlan.zhihu.com/p/497658546
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60 #75
        self.max_paragraph_len = 350 #400
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 300 #350

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
#             mid = int((answer_start_token + answer_end_token) // (2+random.uniform(-1,1)))
#             mid = (answer_start_token + answer_end_token) // 2
#             paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
#             paragraph_end = paragraph_start + self.max_paragraph_len

#             start_min = max(0, answer_end_token - self.max_paragraph_len + 1)
#             start_max = min(answer_start_token, len(tokenized_paragraph) - self.max_paragraph_len)
#             start_max=max(start_min,start_max)
#             paragraph_start = random.randint(start_min, start_max + 1)
#             paragraph_end = paragraph_start + self.max_paragraph_len
            mid = (answer_start_token + answer_end_token) // 2
            answer_length = answer_end_token - answer_start_token + 1
            if answer_length // 2 < self.max_paragraph_len - answer_length // 2:
                rnd = random.randint(answer_length // 2, self.max_paragraph_len - answer_length // 2)
            else:
                rnd = self.max_paragraph_len // 2
            paragraph_start = max(0, min(mid - rnd, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

## Function for Evaluation

In [11]:
#參考資料 :https://blog.csdn.net/qq_43613342/article/details/127044475?spm=1001.2014.3001.5502
def evaluate(data, output,paragraph = None,paragraph_tokenized = None,doc_stride = 300,token_type_ids=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        token_type_id = data[1][0][k].detach().cpu().numpy()
        #print(token_type_id)
        paragraph_start = token_type_id.argmax()
        paragraph_end = len(token_type_id) - 1 - token_type_id[::-1].argmax()-1
        
        if start_index > end_index or start_index < paragraph_start or end_index > paragraph_end:
            continue
#         if start_index > end_index:
#             continue
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if (prob > max_prob) : #and (start_index <= end_index)
            max_prob = prob
            
            old_start = start_index + k * doc_stride - paragraph_start
            old_end = end_index + k * doc_stride - paragraph_start
            
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            
        answer = answer.replace('♡', ' ').replace('△','\u200b').replace('□','\u200e').replace('○','\u3000').replace('*','#').replace(' ','')
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    if '[UNK]' in answer:
        print("出現[UNK]")
        print("預測結果:",answer)
        new_start =  paragraph_tokenized.token_to_chars(old_start)[0]
        new_end = paragraph_tokenized.token_to_chars(old_end)[1]
        answer = paragraph[new_start:new_end]
        print("原本的答案:",answer)
        
    return answer


## Training

In [12]:
from accelerate import Accelerator
from transformers import (get_cosine_schedule_with_warmup)
# hyperparameters
num_epoch = 3 #3
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
train_batch_size = 8

#### TODO: gradient_accumulation (optional)####
# Note: train_batch_size * gradient_accumulation_steps = effective batch size
# If CUDA out of memory, you can make train_batch_size lower and gradient_accumulation_steps upper
# Doc: https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation
gradient_accumulation_steps = 16

# dataloader
# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)


# Change "fp16_training" to True to support automatic mixed 
# precision training (fp16)	
fp16_training = True
if fp16_training:    
    accelerator = Accelerator(mixed_precision="fp16")
else:
    accelerator = Accelerator()

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

nums =num_epoch*len(train_loader)

# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = nums )
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = 100, num_training_steps = nums)

model.train()


print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
           
        train_loss += output.loss
        
        accelerator.backward(output.loss)
        
        step += 1
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        ##### TODO: Apply linear learning rate decay #####

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
          lr = optimizer.state_dict()['param_groups'][0]['lr']
          print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f},learning_rate = {lr}")
          train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output,doc_stride=300,paragraph=dev_paragraphs[dev_questions[i]["paragraph_id"]],paragraph_tokenized=dev_paragraphs_tokenized[dev_questions[i]["paragraph_id"]])== dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 1.519, acc = 0.553,learning_rate = 9.9e-06
Epoch 1 | Step 200 | loss = 0.710, acc = 0.727,learning_rate = 9.997579475390067e-06
Epoch 1 | Step 300 | loss = 0.726, acc = 0.735,learning_rate = 9.990222254887345e-06
Epoch 1 | Step 400 | loss = 0.657, acc = 0.752,learning_rate = 9.977935359458732e-06
Epoch 1 | Step 500 | loss = 0.642, acc = 0.732,learning_rate = 9.960730926920383e-06
Epoch 1 | Step 600 | loss = 0.690, acc = 0.741,learning_rate = 9.938625952960142e-06
Epoch 1 | Step 700 | loss = 0.645, acc = 0.744,learning_rate = 9.911642274348053e-06
Epoch 1 | Step 800 | loss = 0.700, acc = 0.735,learning_rate = 9.87980654736455e-06
Epoch 1 | Step 900 | loss = 0.684, acc = 0.759,learning_rate = 9.843150221467609e-06
Epoch 1 | Step 1000 | loss = 0.580, acc = 0.762,learning_rate = 9.801709508224848e-06
Epoch 1 | Step 1100 | loss = 0.611, acc = 0.762,learning_rate = 9.755525345541348e-06
Epoch 1 | Step 1200 | loss = 0.649, acc = 0.739,learning_rate = 9.704643357218

  0%|          | 0/2863 [00:00<?, ?it/s]

...

出現[UNK]
預測結果: [UNK]紅雀隊
原本的答案: ♡Gas♡House♡紅雀隊
出現[UNK]
預測結果: [UNK]
原本的答案: 鍶
出現[UNK]
預測結果: 香港仔中心;西部的財富花園，華富村，華貴村，數碼港及田灣;南部鴨[UNK]洲;石排灣，香港仔水塘及香港仔郊野公園均為香港仔地區。2011年，香港仔市中心
原本的答案: 香港仔中心♡;西部的財富花園，華富村，華貴村，數碼港及田灣;南部鴨Cha洲;石排灣，香港仔水塘及香港仔郊野公園均為香港仔地區。2011年，香港仔市中心
出現[UNK]
預測結果: [UNK]
原本的答案: UTC
出現[UNK]
預測結果: 1.[UNK]
原本的答案: 1.5M
出現[UNK]
預測結果: 常[UNK]
原本的答案: 常璩
出現[UNK]
預測結果: [UNK]
原本的答案: Opera
出現[UNK]
預測結果: [UNK]
原本的答案: ♡Z♡
出現[UNK]
預測結果: [UNK]東海
原本的答案: JR東海
出現[UNK]
預測結果: 1937年：第一次美國軍艦雷達[UNK]測試在1943年
原本的答案: 1937年：第一次美國軍艦雷達XAF測試在1943年
出現[UNK]
預測結果: [UNK]
原本的答案: C
出現[UNK]
預測結果: [UNK]，達50%
原本的答案: 12931GHD，達50%
出現[UNK]
預測結果: [UNK]
原本的答案: 鉰
出現[UNK]
預測結果: [UNK]
原本的答案: 100°C
Validation | Epoch 2 | acc = 0.913


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 3 | Step 100 | loss = 0.141, acc = 0.926,learning_rate = 2.411164538675743e-06
Epoch 3 | Step 200 | loss = 0.166, acc = 0.917,learning_rate = 2.2780131409696963e-06
Epoch 3 | Step 300 | loss = 0.124, acc = 0.934,learning_rate = 2.147550703731178e-06
Epoch 3 | Step 400 | loss = 0.172, acc = 0.925,learning_rate = 2.0199061064715143e-06
Epoch 3 | Step 500 | loss = 0.139, acc = 0.922,learning_rate = 1.8952054450514507e-06
Epoch 3 | Step 600 | loss = 0.134, acc = 0.926,learning_rate = 1.773571907115233e-06
Epoch 3 | Step 700 | loss = 0.120, acc = 0.938,learning_rate = 1.6551256503976243e-06
Epoch 3 | Step 800 | loss = 0.165, acc = 0.925,learning_rate = 1.5399836840240657e-06
Epoch 3 | Step 900 | loss = 0.160, acc = 0.922,learning_rate = 1.42825975292125e-06
Epoch 3 | Step 1000 | loss = 0.150, acc = 0.920,learning_rate = 1.320064225452276e-06
Epoch 3 | Step 1100 | loss = 0.154, acc = 0.924,learning_rate = 1.2155039843874129e-06
Epoch 3 | Step 1200 | loss = 0.193, acc = 0.906,learning_r

  0%|          | 0/2863 [00:00<?, ?it/s]

出現[UNK]
預測結果: 1566年1月7日至1572年5月1日到位。在羅馬天主教中，他被尊為聖人。[UNK]以其在[UNK]會議中的行動，反宗教改革以及天主教會內秩序的重組而聞名。在他任職期間，他宣布[UNK]是教會教師之一。在外交方面，由於伊麗莎白一世對英國天主教會的迫害和破裂，庇護五世將他驅逐出教堂。庇護五世還組織了一個神聖的聯盟，以對抗奧斯曼帝國對歐洲的襲擊。後來，神聖聯盟在勒班陀戰役中擊敗了奧斯曼帝國海軍，奧斯曼帝國在地中海失去了海上霸權。[UNK]將勝利歸功於聖母瑪利亞的代禱和協助，並下令將神的進步禱告納入聖母瑪利亞的禱告，並於10月7日任命為聖母瑪利亞。[UNK]，前身為[UNK].[UNK]，於1504年出生於米蘭公國的[UNK]市。當祁連利十四歲的時候，我進入了多明戈，我將練習這條路並取名為米歇爾。在1528年
原本的答案: 1566年1月7日至1572年5月1日到位。在羅馬天主教中，他被尊為聖人。Pius♡V以其在Deliton會議中的行動，反宗教改革以及天主教會內秩序的重組而聞名。在他任職期間，他宣布Thomas♡Aquinas是教會教師之一。在外交方面，由於伊麗莎白一世對英國天主教會的迫害和破裂，庇護五世將他驅逐出教堂。庇護五世還組織了一個神聖的聯盟，以對抗奧斯曼帝國對歐洲的襲擊。後來，神聖聯盟在勒班陀戰役中擊敗了奧斯曼帝國海軍，奧斯曼帝國在地中海失去了海上霸權。Pius♡V將勝利歸功於聖母瑪利亞的代禱和協助，並下令將神的進步禱告納入聖母瑪利亞的禱告，並於10月7日任命為聖母瑪利亞。Pius♡V，前身為Antonio♡C.♡Lilian，於1504年出生於米蘭公國的Boso市。當祁連利十四歲的時候，我進入了多明戈，我將練習這條路並取名為米歇爾。在1528年
出現[UNK]
預測結果: [UNK]紅雀隊
原本的答案: ♡Gas♡House♡紅雀隊
出現[UNK]
預測結果: [UNK]
原本的答案: 鍶
出現[UNK]
預測結果: [UNK]
原本的答案: UTC
出現[UNK]
預測結果: 1.[UNK]
原本的答案: 1.5M
出現[UNK]
預測結果: 常[UNK]
原本的答案: 常璩
出現[UNK]
預測結果: [UNK]
原本的答案: Opera
出現[UNK]
預測結果: [UNK]
原本的答案: ♡Z♡
出現[UNK]
預測結果

## Testing

In [13]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate (tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output,doc_stride=300,paragraph=test_paragraphs[test_questions[i]["paragraph_id"]],
                               paragraph_tokenized=test_paragraphs_tokenized[test_questions[i]["paragraph_id"]]))

result_file = "result_COS_100_test_12.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

出現[UNK]
預測結果: [UNK]的本妙寺
原本的答案: Shinjuku的本妙寺
出現[UNK]
預測結果: 回[UNK]部落
原本的答案: 回鶻部落
出現[UNK]
預測結果: 回[UNK]汗國
原本的答案: 回鶻汗國
出現[UNK]
預測結果: 白[UNK]紀中期
原本的答案: 白堊紀中期
出現[UNK]
預測結果: [UNK]大壩
原本的答案: Jawa大壩
出現[UNK]
預測結果: [UNK]
原本的答案: ANP
出現[UNK]
預測結果: 李[UNK]
原本的答案: 李勣
出現[UNK]
預測結果: 每天需要25噸郵票用紙。很多其它的國家則使用木或布料制郵票。例如瑞士就發行過這些郵票，但只面對收藏家。從1955年開始發行郵票的不丹，在1973年4月15日發行了一套68到100毫米的唱片郵票，它們既是郵票又是唱片。蘇聯1965年印有兩枚以航空為主題的鋁郵票。東德在1963年發行了一滌綸郵票，因滌綸一詞[UNK]從發音上非常接近東德的德語首字母縮寫[UNK]。蒲隆地在建國三周年之際發行了金薄膜郵票紀念其獨立。2003年義大利發行牛仔褲料郵票。2004瑞士發行木料郵票。在2000年6月21日，瑞士還發行了世界第一套刺繡郵票。
原本的答案: 每天需要25噸郵票用紙。很多其它的國家則使用木或布料制郵票。例如瑞士就發行過這些郵票，但只面對收藏家。從1955年開始發行郵票的不丹，在1973年4月15日發行了一套68到100毫米的唱片郵票，它們既是郵票又是唱片。蘇聯1965年印有兩枚以航空為主題的鋁郵票。東德在1963年發行了一滌綸郵票，因滌綸一詞Dederon從發音上非常接近東德的德語首字母縮寫DDR。蒲隆地在建國三周年之際發行了金薄膜郵票紀念其獨立。2003年義大利發行牛仔褲料郵票。2004瑞士發行木料郵票。在2000年6月21日，瑞士還發行了世界第一套刺繡郵票。
出現[UNK]
預測結果: 因周王是燕王同母兄弟，而朱允[UNK]怕他與燕王呵成一氣
原本的答案: 因周王是燕王同母兄弟，而朱允炆怕他與燕王呵成一氣
出現[UNK]
預測結果: 久彌宮妃[UNK]子
原本的答案: 久彌宮妃俔子
出現[UNK]
預測結果: 《阿[UNK]婆吠陀》
原本的答案: 《阿闥婆吠陀》
出現[UNK]
預測結果: 姚[UNK]
原本的答案: 姚萇
出現[UNK]
預測結果: 

# GradeScope - Question 2 (In-context learning)

### Try in-context learning
The example prompt is :
```
請從最後一篇的文章中找出最後一個問題的答案：
文章：<文章內容>
問題：<問題敘述>
答案：<答案>
```

In [ ]:
import torch

# To avoid CUDA_OUT_OF_MEMORY
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# You can try model with different size
# When using Colab or Kaggle, model with more than 2 billions parameters may 
# run out of memory
tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-1.7B")

In [ ]:
# for cleaning model output. If you try different prompts, you may have to fix 
# this function on your own
def clean_text(text):
    # Note: When you use unilingual model, the colon may become fullwidth
    text = text.split("答案:")[-1]
    text = text.split(" ")[0]
    return text

In [ ]:
import random
import json
# Change the path of you data
with open("/kaggle/input/2023-ml-hw7-question-answering/hw7_in-context-learning-examples.json", "r") as f: 
    test = json.load(f)

# K-shot learning 
# Give model K examples to make it achieve better accuracy 
# Note: When K >= 4, CUDA_OUT_OFF_MEMORY may occur
K = 1


question_ids = [qa["id"] for qa in test["questions"]]

with torch.no_grad():
    for idx, qa in enumerate(test["questions"]):
        # You can try different prompts
        prompt = "請從最後一篇的文章中找出最後一個問題的答案\n"
        exist_question_indexs = [question_ids.index(qa["id"])]
        # K-shot learning: give the model K examples with answers
        for i in range(K):
            question_index = question_ids.index(qa["id"])
            while(question_index in exist_question_indexs): 
                question_index = random.randint(0, len(question_ids) - 1)
            exist_question_indexs.append(question_index)    
            paragraph_id = test["questions"][question_index]["paragraph_id"]
            prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
            prompt += f'問題：{test["questions"][question_index]["question_text"]}\n'
            prompt += f'答案：{test["questions"][question_index]["answer_text"]}\n'


        # The final one question without answer
        paragraph_id = qa["paragraph_id"]
        prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
        prompt += f'問題：{qa["question_text"]}\n'
        prompt += f'答案：'
        
        inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt") 
        sample = model.generate(**inputs, max_new_tokens = 20)
        text = tokenizer.decode(sample[0], skip_special_tokens=True)
        # Note: You can delete this line to see what will happen
        text = clean_text(text)
        
        print(prompt)
        print(f'正確答案: {qa["answer_text"]}')
        print(f'模型輸出: {text}')
        print()
